In [6]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup

import pandas as pd

import time

from datetime import date
from dateutil.rrule import rrule, DAILY, MONTHLY
from dateutil.relativedelta import relativedelta

import os.path
import os

import sys

In [2]:
detailsList = []

coinDetails = {}
coinDetails['name'] = 'Bitcoin'
coinDetails['start'] = date(2013, 1, 1)
coinDetails['end'] = date(2018, 5, 12)
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Dashpay'
coinDetails['start'] = date(2013, 12, 15)
coinDetails['end'] = date(2018, 5, 12)
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Dogecoin'
coinDetails['start'] = date(2013, 1, 1)
coinDetails['end'] = date(2018, 5, 12)
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Ethereum'
coinDetails['start'] = date(2015, 8, 8)
coinDetails['end'] = date(2018, 5, 12)
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Litecoin'
coinDetails['start'] = date(2013, 5, 5)
coinDetails['end'] = date(2018, 5, 12)
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Monero'
coinDetails['start'] = date(2014, 5, 21)
coinDetails['end'] = date(2018, 5, 12)
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Ripple'
coinDetails['start'] = date(2013, 8, 5)
coinDetails['end'] = date(2018, 5, 12)
detailsList.append(coinDetails)

coinDetails = {}
coinDetails['name'] = 'Stellar'
coinDetails['start'] = date(2014, 8, 5)
coinDetails['end'] = date(2018, 5, 12)
detailsList.append(coinDetails)

In [3]:
profile = webdriver.FirefoxProfile()
profile.set_preference("browser.download.folderList", 2)
profile.set_preference("browser.download.manager.showWhenStarting", False)
profile.set_preference("browser.download.dir", os.getcwd())
profile.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/json, application/x-download")

driver = webdriver.Firefox(firefox_profile=profile)

url = 'https://trends.google.com/trends/explore?date={}%20{}&q={}'

In [5]:
for coinDetail in detailsList:
    print("Scraping {} monthly from Google Trends".format(coinDetail['name']))
    
    old = coinDetail['start']
    coinDetail['start'] = coinDetail['start'] + relativedelta(months=1)
    
    for dt in rrule(MONTHLY, dtstart=coinDetail['start'], until=coinDetail['end']):
        goURL = url.format(old.strftime('%Y-%m-%d'), dt.strftime('%Y-%m-%d'), coinDetail['name'])
        time.sleep(3)
        
        driver.get(goURL)
        
        try:
            elem = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.CSS, 'i.material-icons-extended.gray')))
        except TimeoutException:
            sys.exit("Too much time")
        
        driver.find_elements_by_css_selector('i.material-icons-extended.gray').click()
        time.sleep(2)
        
        os.rename('{}/multiTimeline.csv'.format(os.getcwd()), '{}/{}-{}-{}.csv'.format(os.getcwd(), old.strftime('%Y-%m-%d'), dt.strftime('%Y-%m-%d'), coinDetail['name']))
        
        old  = dt

Scraping Bitcoin monthly from Google Trends
https://trends.google.com/trends/explore?date=2013-02-01%202013-03-01&q=Bitcoin
https://trends.google.com/trends/explore?date=2013-03-01%202013-04-01&q=Bitcoin
https://trends.google.com/trends/explore?date=2013-04-01%202013-05-01&q=Bitcoin
https://trends.google.com/trends/explore?date=2013-05-01%202013-06-01&q=Bitcoin
https://trends.google.com/trends/explore?date=2013-06-01%202013-07-01&q=Bitcoin

KeyboardInterrupt: 

'C:\\Users\\User\\Desktop\\Algo Trading\\CryptoTraderRemote\\CryptoTrader\\data_utils\\trends_data'